In [2]:
import pandas as pd 

In [ ]:
df = pd.read_csv('E:\\MACHINE LEARNING\\datasets\\H1.csv')
df.info()

In [ ]:
df.head(10)

In [ ]:
df.drop(columns=['ReservationStatus'])
df.columns


In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputed = imputer.fit_transform(df[['Country']])
Country = pd.DataFrame(imputed)

dataset = df.join(Country,how='left')

dataset.isnull().sum()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

num_cols = df.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])

num_cols.corr()['IsCanceled'].sort_values(ascending=False)


In [12]:
input_col = dataset[['LeadTime','RequiredCarParkingSpaces','IsRepeatedGuest','PreviousCancellations','BookingChanges','ADR', 'StaysInWeekendNights','StaysInWeekNights','Adults','Children','Babies','PreviousBookingsNotCanceled','DistributionChannel','MarketSegment']]
output_col = dataset[['IsCanceled']]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


categorical_cols = input_col.select_dtypes(include=['object']).columns
numeric_cols = input_col.select_dtypes(exclude=['object']).columns

scaler = StandardScaler()
scaler.fit(input_col[numeric_cols])

encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False,drop='first')
encoder.fit(input_col[categorical_cols])

x = pd.DataFrame(scaler.transform(input_col[numeric_cols]), columns=numeric_cols)
x = pd.concat([x, pd.DataFrame(encoder.transform(input_col[categorical_cols]), columns=encoder.get_feature_names_out(categorical_cols))], axis=1)

x.head(10)


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

x_train,x_test,y_train,y_test = train_test_split(x,output_col,test_size=0.2,random_state=42)


def model_selection(param,model,x_train,y_train):
    grid_search = GridSearchCV(model, param, cv=5,n_jobs=-1)
    grid_search.fit(x_train, y_train)

    y_pred = grid_search.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Best parameters found: ", grid_search.best_params_)
    print('train_score: ', grid_search.best_score_ * 100)
    print('test_score: ', accuracy * 100)




In [ ]:
params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'solver': ['saga'],
    'max_iter': [100, 200, 300],
    'l1_ratio': [0, 0.5, 1] 
}

model_selection(params, LogisticRegression(), x_train, y_train)


logistic regression 76.65 (train) , 77.24 (test)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
params_tree = {
    'max_features': ['sqrt','log2',0.3,0.5,0.7,0.9],
    'max_depth': [None, 5, 10, 15, 20],
    'max_leaf_nodes': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model_selection(params_tree, DecisionTreeClassifier(), x_train, y_train)


decision tree classifier : 80.27 (train) , 80.25 (test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
perams_forest = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
model_selection(perams_forest, RandomForestClassifier(), x_train, y_train)

Random Forest classifier : 83 (train and test both)